In [1]:
import pandas as pd
import igraph as ig
import random
import itertools
import operator
import pickle


In [2]:
##user_id,item_id,timestamp,state_label,comma_separated_list_of_features
#auth = pd.read_csv("./auth_ntlm.txt", header=None, names=['src', 'dst', 'ti', 'lbl', 'src_t', 'dst_t', 'auth_t', 'log_t', 'auth_o', 's_f'])
auth = pd.read_csv("../auth_ntlm.txt", header=None, names=['time', 'src_t', 'user', 'src', 'dst', 'auth_t', 'log_t', 'auth_o', 's_f'])

In [3]:
#auth = auth.reset_index()

In [4]:
authlog = auth[['time', 'user', 'src', 'dst']]

In [5]:
authlog

,time,user,src,dst
0,1,ANONYMOUS LOGON@C586,C1250,C586
1,1,U898@DOM1,C2944,C2944
2,2,ANONYMOUS LOGON@C457,C4615,C457
3,2,ANONYMOUS LOGON@C586,C2734,C586
4,2,ANONYMOUS LOGON@C586,C3896,C586
...,...,...,...,...
49341295,5011199,C659$@DOM1,C659,C1065
49341296,5011199,C9458$@DOM1,C9458,C1015
49341297,5011199,U129@DOM1,C419,C419
49341298,5011199,U3887@DOM1,C9458,C1015


## 1- Build In Host User Map

In [6]:
def BuildInHostUserMap(authlog):
    InHostUserMap = {}
    for index, event in authlog.iterrows():
        if event['dst'] not in InHostUserMap:
            InHostUserMap[event['dst']] = {}
        if event['user'] not in InHostUserMap[event['dst']]:
            InHostUserMap[event['dst']][event['user']] = {} 
        day = event['time']/86400
        if day not in InHostUserMap[event['dst']][event['user']]:
             InHostUserMap[event['dst']][event['user']][day] = 0 
        InHostUserMap[event['dst']][event['user']][day] += 1
        
    return InHostUserMap

In [7]:
InHostUserMap = BuildInHostUserMap(authlog)
pickle.dump(InHostUserMap, open("InHostUserMap.pkl", "wb")) 

## 2- Build In Host Src Map

In [8]:
def BuildInHostSrcMap(authlog):
    InHostSrcMap = {}
    for index, event in authlog.iterrows():
        if event['dst'] not in InHostSrcMap:
            InHostSrcMap[event['dst']] = {}
        if event['src'] not in InHostSrcMap[event['dst']]:
            InHostSrcMap[event['dst']][event['src']] = {} 
        day = event['time']/86400
        if day not in InHostSrcMap[event['dst']][event['src']]:
             InHostSrcMap[event['dst']][event['src']][day] = 0
        InHostSrcMap[event['dst']][event['src']][day] += 1
        
    return InHostSrcMap

In [9]:
InHostSrcMap = BuildInHostSrcMap(authlog)
pickle.dump(InHostSrcMap, open("InHostSrcMap.pkl", "wb")) 

## 3- Build In Host Usr Src Map

In [10]:
def BuildInHostUsrSrcMap(authlog):
    InHostUsrSrcMap = {}
    for index, event in authlog.iterrows():
        if event['dst'] not in InHostUsrSrcMap:
            InHostUsrSrcMap[event['dst']] = {}
        if event['user']+event['src'] not in InHostUsrSrcMap[event['dst']]:
            InHostUsrSrcMap[event['dst']][event['user']+event['src']] = {} 
        day = event['time']/86400
        if day not in InHostUsrSrcMap[event['dst']][event['user']+event['src']]:
             InHostUsrSrcMap[event['dst']][event['user']+event['src']][day] = 0
        InHostUsrSrcMap[event['dst']][event['user']+event['src']][day] += 1
        
    return InHostUsrSrcMap

In [11]:
InHostUsrSrcMap = BuildInHostUsrSrcMap(authlog)
pickle.dump(InHostUsrSrcMap, open("InHostUsrSrcMap.pkl", "wb")) 

## 4- Build Out Host Usr Map

In [12]:
def BuildOutHostUsrMap(authlog):
    OutHostUserMap = {}
    for index, event in authlog.iterrows():
        if event['src'] not in OutHostUserMap:
            OutHostUserMap[event['src']] = {}
        if event['user'] not in OutHostUserMap[event['src']]:
            OutHostUserMap[event['src']][event['user']] = {} 
        day = event['time']/86400
        if day not in OutHostUserMap[event['src']][event['user']]:
             OutHostUserMap[event['src']][event['user']][day] = 0
        OutHostUserMap[event['src']][event['user']][day] += 1
        
    return OutHostUserMap

In [ ]:
OutHostUserMap = BuildOutHostUsrMap(authlog)
pickle.dump(OutHostUserMap, open("OutHostUserMap.pkl", "wb")) 

## 5- Build Out Host Dst Map

In [ ]:
def BuildOutHostDstMap(authlog):
    OutHostDstMap = {}
    for index, event in authlog.iterrows():
        if event['src'] not in OutHostDstMap:
            OutHostDstMap[event['src']] = {}
        if event['dst'] not in OutHostDstMap[event['src']]:
            OutHostDstMap[event['src']][event['dst']] = {} 
        day = event['time']/86400
        if day not in OutHostDstMap[event['src']][event['dst']]:
             OutHostDstMap[event['src']][event['dst']][day] = 0
        OutHostDstMap[event['src']][event['dst']][day] += 1
        
    return OutHostDstMap

In [ ]:
OutHostDstMap = BuildOutHostDstMap(authlog)
pickle.dump(OutHostDstMap, open("OutHostDstMap.pkl", "wb")) 

## 6- Build Out Host Usr Dst Map

In [ ]:
def BuildOutHostUsrDstMap(authlog):
    OutHostUsrDstMap = {}
    for index, event in authlog.iterrows():
        if event['src'] not in OutHostUsrDstMap:
            OutHostUsrDstMap[event['src']] = {}
        if event['user']+event['dst'] not in OutHostUsrDstMap[event['src']]:
            OutHostUsrDstMap[event['src']][event['user']+event['dst']] = {} 
        day = event['time']/86400
        if day not in OutHostUsrDstMap[event['src']][event['user']+event['dst']]:
             OutHostUsrDstMap[event['src']][event['user']+event['dst']][day] = 0
        OutHostUsrDstMap[event['src']][event['user']+event['dst']][day] += 1
        
    return OutHostUsrDstMap

In [ ]:
OutHostUsrDstMap = BuildOutHostUsrDstMap(authlog)
pickle.dump(OutHostUsrDstMap, open("OutHostUsrDstMap.pkl", "wb")) 